In [2]:
# Fixed dependencies - do not remove or change.
import pytest
import pandas as pd
import numpy as np
import datetime

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.impute import SimpleImputer
import pickle
from structure import horse
import structure
import os
import random

import pandas as pd

#import matplotlib.pyplot as plt
import tensorflow as tf



races = []



def loadAll():
    global races
    files = os.listdir("pickles/")
    for file in files:
        filename =  "pickles/{}".format(file)
        try:
            with open(filename, "rb") as f:
                thisRace = pickle.load(f)
                races.append(thisRace)
        except:
            pass



def horseCount():
    loadAll()
    horseCount = 0
    global races
    for race in races:
        horseCount += len(race.horses)
    print(horseCount, "horses")

def getRace(raceId):
    files = os.listdir("pickles/")
    for file in files:
        
        if str(raceId) in file:
            filename =  "pickles/{}".format(file)
            with open(filename, "rb") as f:
                thisRace = pickle.load(f)
                return thisRace

In [4]:
# SAVE ALL INTO allRaces.pickle
def loadAllSave():
    global races
    loadAll()
    with open("allRaces.pickle", 'wb') as f:
        pickle.dump(races, f)

def loadAllRaces():
    print("Loading All Data")
    global races
    races = []
    with open("allRaces.pickle", "rb") as f:
        races = pickle.load(f)
    print("All Data Loaded")

loadAllRaces()

Loading All Data
All Data Loaded


In [4]:
loadAll()

print(len(races))

Loading All Data
All Data Loaded
47164


In [ ]:
# SAVE ALL RACES

files = os.listdir("pickles/")
races = []


def loadAll():
    global races
    global files
    for file in files:
        filename =  "pickles/{}".format(file)
        try:
            with open(filename, "rb") as f:
                thisRace = pickle.load(f)
                races.append(thisRace)
        except:
            pass

loadAll()

for race in races:
    race.save()

In [ ]:
# INSPECTOR

files = os.listdir("pickles/")
races = []


def loadAll():
    global races
    global files
    for file in files:
        filename =  "pickles/{}".format(file)
        try:
            with open(filename, "rb") as f:
                thisRace = pickle.load(f)
                races.append(thisRace)
        except:
            pass

loadAll()
data = []
for race in races:
    d = race.getAll()
    for line in d:
        data.append(line)

raw_data = pd.DataFrame(data)

raw_data.to_csv("inspect.csv")

In [ ]:
# Print specific race

raceFilename = "pickles/SL677963.pickle"
thisRace = None
with open(raceFilename, "rb") as f:
    thisRace = pickle.load(f)

print(thisRace.raceURL)
for h in thisRace.horses:

    print(h.name, h.age, h.form, h.weight)
    print()


In [22]:


#tomorrow = "2022-{:02d}-{:02d}".format(month, day)
tomorrow = "2022-05-01"
loadAll()
data = []
for race in races:
    race.fixData()
    if tomorrow not in race.date:
        d = race.getData()
        for line in d:
            for i in range(0, len(line)):
                if line[i] == "-":
                    line[i] = None
            data.append(line)
tomorrowsLength = len(data)

for race in races:
    race.fixData()
    if tomorrow in race.date:
        d = race.getData()
        for line in d:
            for i in range(0, len(line)):
                if line[i] == "-":
                    line[i] = None
            data.append(line)

print(tomorrowsLength)
print(len(data))

raw_data = pd.DataFrame(data)
originalData = data[tomorrowsLength:]
raw_data.to_csv("final.csv")

422040
423276


In [ ]:
print(data[12])

In [23]:
raw_data = pd.DataFrame(data)

columnsToEncode = [0, 2, 3, 4, 5, 6, 8]
le = LabelEncoder()
for column in columnsToEncode:
    first_col = raw_data.pop(column)
    first_col = le.fit_transform(first_col)
    raw_data.insert(0, column,first_col)

#raw_data.to_csv("finalCheck.csv")


x = raw_data.iloc[:, :-1].values
y = raw_data.iloc[:, -1:].values

for i in range(len(y)):
    if y[i] < 2:
        y[i] = 1.0
    else:
        y[i] = 0.0

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(x)
x = imputer.transform(x)



#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = todaysLength)
#originalXTest = x_test

#todaysLength

sc = StandardScaler()
#x_train = sc.fit_transform(x_train)
#x_test = sc.transform(x_test)

x = sc.fit_transform(x)
x_train = x[:tomorrowsLength]
x_pred = x[tomorrowsLength:]

#y = sc.fit_transform(y)
y_train = y[:tomorrowsLength]
y_pred = y[tomorrowsLength:]



ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=18, activation="relu"))
ann.add(tf.keras.layers.Dense(units=18, activation="relu"))
ann.add(tf.keras.layers.Dense(units=18, activation="relu"))
ann.add(tf.keras.layers.Dense(units=18, activation="relu"))
ann.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
ann.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
ann.fit(x_train, y_train, batch_size = 32, epochs = 1)


13189/13189 [==============================] - 34s 3ms/step - loss: 0.3308 - accuracy: 0.8892


In [ ]:
results = ann.predict(x_pred)
for i in range(len(results)):
    print(originalData[i][-1], results[i], y_pred[i])

In [32]:
wins = 0
fails = 0

moneyMade = 0
moneyLost = 0

for i in range(0, len(results)):
    if results[i] > 0.1:
        print(originalData[i][-1], results[i], y_pred[i])
        if y_pred[i] == 0:
            fails += 1
            moneyLost += 1
        else:
            wins += 1
            try:
                moneyMade += float(originalData[i][-7])
            except:
                pass
print(len(results), wins, fails)#, fails/wins)
print(moneyMade, moneyLost)
print(moneyMade-moneyLost)

2 [0.12453313] [0]
3 [0.15685023] [0]
7 [0.11371591] [0]
1 [0.15762605] [1]
2 [0.18863674] [0]
3 [0.20639971] [0]
14 [0.11789061] [0]
1 [0.23158225] [1]
2 [0.10432822] [0]
3 [0.12875846] [0]
4 [0.11703] [0]
1 [0.23538771] [1]
2 [0.1529631] [0]
3 [0.13173956] [0]
1 [0.20504999] [1]
2 [0.32507485] [0]
3 [0.13794796] [0]
1 [0.2318036] [1]
8 [0.18786608] [0]
1 [0.26536468] [1]
2 [0.37608713] [0]
3 [0.22202209] [0]
1 [0.35946372] [1]
2 [0.11816545] [0]
1 [0.2524806] [1]
2 [0.28786194] [0]
3 [0.3649084] [0]
4 [0.10263711] [0]
3 [0.10719224] [0]
2 [0.20567624] [0]
3 [0.10445807] [0]
5 [0.10888847] [0]
6 [0.11376559] [0]
7 [0.10916971] [0]
1 [0.28607032] [1]
2 [0.1578124] [0]
3 [0.10064272] [0]
2 [0.2966369] [0]
1 [0.19648607] [1]
2 [0.10476189] [0]
3 [0.20202] [0]
4 [0.10317653] [0]
5 [0.13271582] [0]
1 [0.19323215] [1]
2 [0.36370108] [0]
3 [0.32946074] [0]
5 [0.19235568] [0]
2 [0.11500756] [0]
3 [0.13478263] [0]
11 [0.16824591] [0]
1 [0.13334034] [1]
2 [0.21733497] [0]
3 [0.30792776] [0]
4 [